# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

house = pd.read_csv('CaliforniaHousingData.csv')

house = house.drop(house.columns[0], axis=1)
house.head()

,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedInc,Latitude,Longitude,medianHouseValue
0,41.0,6.984127,1.023810,322.0,2.555556,8.3252,37.88,-122.23,4.526
1,21.0,6.238137,0.971880,2401.0,2.109842,8.3014,37.86,-122.22,3.585
2,52.0,8.288136,1.073446,496.0,2.802260,7.2574,37.85,-122.24,3.521
3,52.0,5.817352,1.073059,558.0,2.547945,5.6431,37.85,-122.25,3.413
4,52.0,6.281853,1.081081,565.0,2.181467,3.8462,37.85,-122.25,3.422


In [2]:
ydata = house['medianHouseValue']
xdata = house.drop('medianHouseValue',axis= 1)
xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size=0.3)
house.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
HouseAge,20640.0,28.639486,12.585558,1.000000,18.000000,29.000000,37.000000,52.000000
AveRooms,20640.0,5.429000,2.474173,0.846154,4.440716,5.229129,6.052381,141.909091
AveBedrms,20640.0,1.096675,0.473911,0.333333,1.006079,1.048780,1.099526,34.066667
Population,20640.0,1425.476744,1132.462122,3.000000,787.000000,1166.000000,1725.000000,35682.000000
AveOccup,20640.0,3.070655,10.386050,0.692308,2.429741,2.818116,3.282261,1243.333333
MedInc,20640.0,3.870671,1.899822,0.499900,2.563400,3.534800,4.743250,15.000100
Latitude,20640.0,35.631861,2.135952,32.540000,33.930000,34.260000,37.710000,41.950000
Longitude,20640.0,-119.569704,2.003532,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000
medianHouseValue,20640.0,2.068558,1.153956,0.149990,1.196000,1.797000,2.647250,5.000010


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [3]:
scal = MinMaxScaler()
scal.fit(xtrain)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [4]:
xtrain = pd.DataFrame (data=scal.transform(xtrain), columns = xtrain.columns, index=xtrain.index)
xtest = pd.DataFrame  (data=scal.transform(xtest),  columns = xtest.columns,  index=xtest.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [5]:
house.columns

Index(['HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'MedInc',
       'Latitude', 'Longitude', 'medianHouseValue'],
      dtype='object')

In [6]:
age =        tf.feature_column.numeric_column('HouseAge')
rooms =      tf.feature_column.numeric_column('AveRooms')
bedrooms =   tf.feature_column.numeric_column('AveBedrms')
population = tf.feature_column.numeric_column('Population')
occupation = tf.feature_column.numeric_column('AveOccup')
income =     tf.feature_column.numeric_column('MedInc')
latitude =   tf.feature_column.numeric_column('Latitude')
longitude =  tf.feature_column.numeric_column('Longitude')

In [7]:
featurecolunms = [age, rooms, bedrooms, population, occupation, income, latitude, longitude]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [8]:
input_func = tf.estimator.inputs.pandas_input_fn(x=xtrain, y=ytrain, batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [9]:
model = tf.estimator.DNNRegressor(hidden_units=[10, 10, 10],feature_columns=featurecolunms)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\1\\AppData\\Local\\Temp\\tmpix3s6vq5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020F24103DA0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [10]:
model.train(input_fn=input_func, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\1\AppData\Local\Temp\tmpix3s6vq5\model.ckpt.
INFO:tensorflow:loss = 38.469673, step = 1
INFO:tensorflow:global_step/sec: 285.37
INFO:tensorflow:loss = 18.658415, step = 101 (0.349 sec)
INFO:tensorflow:global_step/sec: 429.391
INFO:tensorflow:loss = 2.5066974, step = 201 (0.222 sec)
INFO:tensorflow:global_step/sec: 450.707
INFO:tensorflow:loss = 20.099045, step = 301 (0.220 sec)
INFO:tensorflow:global_step/sec: 495.335
INFO:tensorflow:loss = 3.9618464, step = 401 (0.200 sec)
INFO:tensorflow:global_step/sec: 429.428
INFO:tensorflow:loss = 2.1260457, step = 501 (0.236 sec)
INFO:tensorflow:global_step/sec: 420.407
INFO:tensorflow:loss = 5.351326, step = 601 (0.238 sec)
INFO:tensorflow:global_step/sec: 3

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [11]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=xtest, batch_size=10, num_epochs=1, shuffle=False)
pred_gen = model.predict(predict_input_func)
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\1\AppData\Local\Temp\tmpix3s6vq5\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 1 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [17]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

mean_squared_error(ytest,final_preds)**0.5

0.7599744510384301

# Great Job!